In [1]:
import pandas as pd
import gzip

def parse(path):
  g = gzip.open(path, 'rb')
  for l in g:
    yield eval(l)

def getDF(path):
  i = 0
  df = {}
  for d in parse(path):
    df[i] = d
    i += 1
  return pd.DataFrame.from_dict(df, orient='index')

df = getDF('reviews_Automotive_5.json.gz')

C:\Users\prasa\anaconda3\lib\site-packages\pandas\core\computation\expressions.py:21: UserWarning: Pandas requires version '2.7.3' or newer of 'numexpr' (version '2.7.1' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED


In [2]:
df.head()

,reviewerID,asin,reviewerName,helpful,reviewText,overall,summary,unixReviewTime,reviewTime
0,A3F73SC1LY51OO,B00002243X,Alan Montgomery,"[4, 4]",I needed a set of jumper cables for my new car...,5.0,Work Well - Should Have Bought Longer Ones,1313539200,"08 17, 2011"
1,A20S66SKYXULG2,B00002243X,alphonse,"[1, 1]","These long cables work fine for my truck, but ...",4.0,Okay long cables,1315094400,"09 4, 2011"
2,A2I8LFSN2IS5EO,B00002243X,Chris,"[0, 0]",Can't comment much on these since they have no...,5.0,Looks and feels heavy Duty,1374710400,"07 25, 2013"
3,A3GT2EWQSO45ZG,B00002243X,DeusEx,"[19, 19]",I absolutley love Amazon!!! For the price of ...,5.0,Excellent choice for Jumper Cables!!!,1292889600,"12 21, 2010"
4,A3ESWJPAVRPWB4,B00002243X,E. Hernandez,"[0, 0]",I purchased the 12' feet long cable set and th...,5.0,"Excellent, High Quality Starter Cables",1341360000,"07 4, 2012"


In [3]:
# df = df[['reviewerID', 'asin', 'reviewerName', 'reviewText', 'overall','summary']]
df.reviewText.fillna('', inplace=True)
# df['description'] = df['tagline'].map(str) + ' ' + df['overview']
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20260 entries, 0 to 20472
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   reviewerID      20260 non-null  object 
 1   asin            20260 non-null  object 
 2   reviewerName    20260 non-null  object 
 3   helpful         20260 non-null  object 
 4   reviewText      20260 non-null  object 
 5   overall         20260 non-null  float64
 6   summary         20260 non-null  object 
 7   unixReviewTime  20260 non-null  int64  
 8   reviewTime      20260 non-null  object 
dtypes: float64(1), int64(1), object(7)
memory usage: 1.5+ MB


In [15]:
df[['overall', 'asin']].groupby(['asin']).value_counts()

asin        overall
B00002243X  5.0        8
            4.0        2
B00002243Z  4.0        4
            5.0        2
B00008BKX5  5.0        4
                      ..
B00JGP8I1S  4.0        2
B00KIAQ8VW  5.0        6
            2.0        2
            4.0        1
            3.0        1
Length: 4906, dtype: int64

In [22]:
(df['asin'] == 'B00002243X').sum()

10

In [12]:
df['overall'].value_counts()

5.0    13787
4.0     3927
3.0     1413
2.0      597
1.0      536
Name: overall, dtype: int64

In [4]:
import nltk
import re
import numpy as np

stop_words = nltk.corpus.stopwords.words('english')

def normalize_document(doc):
    # lower case and remove special characters\whitespaces
    doc = re.sub(r'[^a-zA-Z0-9\s]', '', doc, re.I|re.A)
    doc = doc.lower()
    doc = doc.strip()
    # tokenize document
    tokens = nltk.word_tokenize(doc)
    # filter stopwords out of document
    filtered_tokens = [token for token in tokens if token not in stop_words]
    # re-create document from filtered tokens
    doc = ' '.join(filtered_tokens)
    return doc

normalize_corpus = np.vectorize(normalize_document)

norm_corpus = normalize_corpus(list(df['reviewText']))
len(norm_corpus)

20260

In [11]:
norm_corpus

array(['needed set jumper cables new car good reviews good price used times already supposed complaints therewhat say 12 feet really isnt ideal length sure pull front bumper front bumper plenty long lot times beside another car cant get really close would recommend something little longer 12great brand get 16 version though',
       'long cables work fine truck quality seems little shabby side money expecting 200 dollar snapon jumper cables seem like would see chinese knock shop like harbor freight 30 bucks',
       'cant comment much since yet used come back update review find issues first use built solid nice tough big hard clamps love long cable never move cars around anything needed bought new truck since always need cables another reason tow travel trailer run batteries shore power may die couple days need quick small recharge perfect length pop hood run cables back truck hook batteries tongue travel trailer running truck 3045 minutes nice large gauge wire connected battery tot tr

In [7]:
from sklearn.feature_extraction.text import TfidfVectorizer


tf = TfidfVectorizer(ngram_range=(1, 2), min_df=2)
tfidf_matrix = tf.fit_transform(norm_corpus)
tfidf_matrix = tfidf_matrix.toarray()
vocab = tf.get_feature_names()
pd.DataFrame (np.round(tfidf_matrix, 2), columns = vocab)
tfidf_matrix.shape

(20260, 115867)

In [11]:
from sklearn.metrics.pairwise import cosine_similarity

doc_sim = cosine_similarity(tfidf_matrix)
doc_sim_df = pd.DataFrame(doc_sim)
doc_sim_df.head()

,0,1,2,3,4,5,6,7,8,9,...,20250,20251,20252,20253,20254,20255,20256,20257,20258,20259
0,1.000000,0.075296,0.036320,0.080788,0.070488,0.083307,0.046316,0.026017,0.000000,0.016370,...,0.000000,0.003959,0.000000,0.000000,0.000000,0.000000,0.018239,0.009784,0.007793,0.007001
1,0.075296,1.000000,0.064049,0.078815,0.106873,0.150348,0.043456,0.000000,0.009713,0.016958,...,0.000000,0.012938,0.000000,0.000000,0.000000,0.005540,0.025009,0.000000,0.011556,0.010646
2,0.036320,0.064049,1.000000,0.127618,0.070832,0.113439,0.059027,0.018355,0.072625,0.109039,...,0.000000,0.006450,0.006957,0.004550,0.003141,0.013590,0.012143,0.011159,0.004983,0.000000
3,0.080788,0.078815,0.127618,1.000000,0.093464,0.171662,0.085736,0.043613,0.067399,0.049353,...,0.011424,0.009586,0.005516,0.016181,0.005617,0.005621,0.030724,0.016760,0.015712,0.012260
4,0.070488,0.106873,0.070832,0.093464,1.000000,0.118312,0.045347,0.000000,0.031431,0.032223,...,0.000000,0.000000,0.000000,0.011637,0.000000,0.000000,0.027302,0.000000,0.016148,0.000000
